In [1]:
from hypex.dataset.dataset import Dataset, ExperimentData
from hypex.dataset.roles import (
    InfoRole,
    FeatureRole,
    TreatmentRole,
    PreTargetRole,
    TargetRole,
)
from hypex.experiments.aa_test import AA_TEST
from hypex.reporters.aa import AADictReporter

In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(),
        "signup_month": FeatureRole(),
        "treat": TreatmentRole(),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole(),
        "age": TargetRole(),
        "gender": FeatureRole(),
        "industry": FeatureRole(),
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data.roles

{'user_id': Info,
 'signup_month': Feature,
 'treat': Treatment,
 'pre_spends': Target,
 'post_spends': Target,
 'age': Target,
 'gender': Feature,
 'industry': Feature}

In [4]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [5]:
result.analysis_tables['OneAASplitAnalyzer╰╰']

   TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
0       0.173506         0.0        0.460366          0.0         0.364746

In [6]:
AADictReporter().report(result)

{'random_state': None,
 'B pre_spends control mean': 487.3426,
 'B pre_spends difference': -0.4977000000000089,
 'B pre_spends difference %': -0.10212528106511298,
 'B pre_spends test mean': 486.8449,
 'B post_spends control mean': 452.7185777777778,
 'B post_spends difference': -1.108044444444488,
 'B post_spends difference %': -0.24475347353392074,
 'B post_spends test mean': 451.6105333333333,
 'B age control mean': 43.54577777777778,
 'B age difference': 0.21733333333332894,
 'B age difference %': 0.49909163281551816,
 'B age test mean': 43.76311111111111}

In [7]:
result.analysis_tables['GroupDifference╰╰post_spends[A]'].to_dict()

{'backend': 'pandas',
 'roles': {'role_names': ['B'], 'columns': [Statistic]},
 'data': {'data': {'B': [452.7185777777778,
    -1.108044444444488,
    -0.24475347353392074,
    451.6105333333333]},
  'index': ['post_spends control mean',
   'post_spends difference',
   'post_spends difference %',
   'post_spends test mean']}}